In [1]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import bs4
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Path of WedbDriver
driver = webdriver.Chrome('C:/Users/zheny/Downloads/chromedriver')

# Open Webpage Link
driver.get("https://www.glassdoor.sg/member/home/index.htm")

# Locate username and password searchbox
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
                                                                       "input[name='username']")))
username.clear()
username.send_keys("zhenyuen_168_2009@hotmail.com") # fill in username here

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
time.sleep(10)
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
                                                                       "input[name='password']")))
password.clear()
password.send_keys("ngzhyuen5") # fill in password here

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
time.sleep(10)


C:\Users\zheny\AppData\Local\Temp/ipykernel_4924/3457204276.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/zheny/Downloads/chromedriver')


In [10]:
def extract_data():
    
    # Get the page source
    source = driver.page_source
    test_soup = bs4.BeautifulSoup(source)
    soup = bs4.BeautifulSoup(source, 'html.parser')
    
    yearList = []
    dateList = []
    scoreList = []
    prosList = []
    consList = []
    titleList = []
    
    article_list = test_soup.find_all("li", class_="noBorder empReview cf pb-0 mb-0")
    count = 0
    continue_reading = driver.find_elements(By.CLASS_NAME, 'v2__EIReviewDetailsV2__continueReading ')
    for x in range(0, len(continue_reading)):
        time.sleep(1)
        try:
            if continue_reading[x].is_displayed():
                continue_reading[x].click()
                
        except NameError as e:
            continue

    for article in article_list:
        title = article.find('a', {'class': 'reviewLink'}).text
        score = article.find("span", {'class':'ratingNumber mr-xsm'}).text
        yearexp = article.find("span", class_='pt-xsm pt-md-0 css-1qxtz39 eg4psks0').text
        date = article.find("span", class_='authorJobTitle middle common__EiReviewDetailsStyle__newGrey').text
        pros = article.find("span", {"data-test":'pros'}).text
        cons = article.find("span", {"data-test":'cons'}).text

        titleList.append(title)
        yearList.append(yearexp)
        dateList.append(date)
        scoreList.append(score)
        prosList.append(pros)
        consList.append(cons)

    dataDict={
        'headline': titleList,
        "date": dateList,
        "year exp": yearList,
        "score": scoreList,
        "pros": prosList,
        "cons": consList
    }
    
    dataDf = pd.DataFrame(dataDict)
#     print(dataDf)
    return dataDf

In [12]:
links_to_scrape = ['https://www.glassdoor.sg/Reviews/TikTok-Reviews-E2230881_P', 
                   'https://www.glassdoor.sg/Reviews/Shopee-Reviews-E1263091_P',
                   'https://www.glassdoor.sg/Reviews/Procter-and-Gamble-Reviews-E544_P',
                   'https://www.glassdoor.sg/Reviews/Meta-Reviews-E40772_P',
                   'https://www.glassdoor.sg/Reviews/Google-Reviews-E9079_P',
                   'https://www.glassdoor.sg/Reviews/American-Express-Reviews-E35_P', 
                   'https://www.glassdoor.sg/Reviews/Certis-Reviews-E309683_P',
                   'https://www.glassdoor.sg/Reviews/Visa-Inc-Reviews-E3035_P',
                   'https://www.glassdoor.sg/Reviews/Walmart-Reviews-E715_P']

In [13]:
def get_company_name(txt):
    txt = txt.partition('Reviews/')[2]
    txt = txt.partition('-Reviews')[0]
    txt = txt.replace('-', ' ')
    return txt

In [14]:
# link = 'https://www.glassdoor.sg/Reviews/ByteDance-Reviews-E1624196_P'
back_link = '.htm?filter.iso3Language=eng'

In [17]:
count = 0
for links in links_to_scrape:
    name = get_company_name(links)
    
    for i in range(1, 20):
        driver.get(links + str(i) + back_link)
        data = extract_data()
        data['company'] = name
        print(data)
        if count == 0:
            full_df = data
        else:
            full_df = pd.concat([full_df, data])
            
        count += 1

In [18]:
full_df.head()

,headline,date,year exp,score,pros,cons,company
0,"Big problem to solve, big chance to learn and ...",1 Nov 2022 - Platform Engineer,"Current Employee, more than 1 year",5.0,"Great culture, high payment, very nice college.","Learning quickly to solve entire new problem ,...",TikTok
1,Great Place,29 Oct 2022 - Anonymous Employee,Current Employee,5.0,Very great great great great,nothing much to say say,TikTok
2,"well pay, work overtime",27 Oct 2022 - Software Development Engineer (SDE),Current Employee,4.0,"learn new tech, high salary","overwork, not WLB, cross region work",TikTok
3,Not too bad,20 Oct 2022 - Anonymous Employee,Current Employee,4.0,Flexible working environment. . . .,. . . . .,TikTok
4,Finding your spot,19 Oct 2022 - Project Manager,"Current Employee, less than 1 year",4.0,- Ample learning opportunities - Many teams sp...,- flat hierarchy which makes it hard to unders...,TikTok


In [19]:
full_df.to_csv('company_glassdoor_data.csv')